# TorchVision SSD Finetuning Tutorial

Based on [TorchVision Object Detection Finetuning Tutorial](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html).

## Defining your model

In [ ]:
from torchvision.models.detection.ssd import ssd300_vgg16, SSD300_VGG16_Weights


model = ssd300_vgg16(weights=SSD300_VGG16_Weights.DEFAULT)

## Training and evaluation functions

In [ ]:
import lib.detection.transforms as T
from lib.penn_fundan import PennFudanDataset

# Define data transforms for training batches
train_tfm = T.Compose([
    T.ToTensor(),  # converts the image, a PIL image, into a PyTorch Tensor
    T.RandomHorizontalFlip(0.5)  # randomly flip the training images
])

# Define data transforms for validation batches
val_tfm = T.ToTensor()

# Define datasets
dataset_train = PennFudanDataset('data/PennFudanPed/', train_tfm)
dataset_val = PennFudanDataset('data/PennFudanPed/', val_tfm)

In [ ]:
from torch.utils.data import DataLoader
from lib.detection.utils import collate_fn


# define training and validation data loaders
data_loader_train = DataLoader(
    dataset_train, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=collate_fn
)

data_loader_val = DataLoader(
    dataset_val, batch_size=2, shuffle=False, num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
import torch
from torch.optim.lr_scheduler import StepLR
from torch.optim import SGD


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and product
num_classes = 2

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = SGD(params, lr=0.0005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = StepLR(optimizer,
                      step_size=3,
                      gamma=0.1)

## Start TensorBoard for logging

To start TensorBoard on VSC OnDemand, go to [the dashboard](https://ondemand.hpc.kuleuven.be/pun/sys/dashboard/) and click on "TensorBoard". Use the following settings:

- Number of cores: 1
- Account: lp_edu_maibi_anndl
- Partition: interactive
- Project/Log folder: maibi_cv/3_detection/runs
- Number of hours: 4
- Number of gpu's: 0

Leave the other settings at their default values.

In [ ]:
from lib.detection.engine import train_one_epoch, evaluate
from torch.utils.tensorboard import SummaryWriter

num_epochs = 10

writer = SummaryWriter()

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader_train,
                    device, epoch, writer=writer)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the validation dataset
    evaluate(model, data_loader_val, device, epoch, writer=writer)

In [ ]:
# pick one image from the test set
img, _ = dataset_val[0]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

In [ ]:
prediction

In [ ]:
from PIL import Image, ImageDraw

im = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
draw = ImageDraw.Draw(im)

for box in prediction[0]['boxes'].cpu().numpy():
    draw.rectangle(box, width=5)

im